In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();
using ForwardDiff
using Test
using RobotZoo: PlanarQuadrotor
using RobotDynamics
using LinearAlgebra
using StaticArrays
using SparseArrays
using Plots
using Printf

include("quadratic_cost.jl")


 Activating environment at `~/Desktop/Home/CMU/2021 Courses/16745 OC/hw3-mfogelson/Project.toml`


run_tests (generic function with 1 method)

# Final Project Info
In this we present a geostationary transfer orbit to a geosynchronous (GSO) or geostationary orbit (GEO).  We use Keplerian Orbital Dynamics to model our system. We make several assumptions like Earth Reference Frame, Spherical Earth, Kepler Dynamics. 

## The Model
We'll be using the Kepler Dynamics:

$\newcommand{\norm}[1]{\left\lVert#1\right\rVert}$

$$ 
\ddot{r} = F = \frac{-\mu}{\norm{r}^3}*r+u \\
x = [r, \dot{r}] \\
\norm{u} \le Tmax
$$
with the following model parameters:
* $\mu$: Gravitational Parameter = 63781
* $r$: distance to earth center
* $u$: control of ion thruster

In [2]:
# Problem Setup
model = Kepler() # TODO
n,m = size(model)          # state and control dimensions
dt = 0.025                 # time step (sec)
tf = 1.5                   # time horizon (sec)
T = Int(tf/dt) + 1         # number of time steps

# Initial & final condition
x0    = #TODO                    # initial state
xgoal = #TODO                    # goal state
ugto = #TODO                     # hover controls

# Cost matrices
Q = #TODO  #Diagonal(SVector{6}([ones(3) ; fill(0.1, 3)]))
R = #TODO  #Diagonal(@SVector fill(1e-2, m))
Qf = #TODO #Diagonal(@SVector fill(1e2, n));

## Part (a): Implement DIRCOL 
Implement DIRCOL by completing the provided methods. 

In [3]:
"""
    Problem{n,m,L}

Describes a trajectory optimization problem with `n` states, `m` controls, and 
a model of type `L`. 

# Constructor
    Problem(model::L, obj::Vector{<:QuadraticCost{n,m}}, tf, x0, xf) where {n,m,L}

where `tf` is the final time, and `x0` is the initial state. 
"""
struct Problem{n,m,L}
    # TODO 
end
Base.size(prob::Problem{n,m}) where {n,m} = (n,m,prob.T)

In [4]:
# Cost function
# TODO 

# Build Problem
prob = Problem(model, obj, tf, x0);
n,m,T = size(prob)   # get problem dimensions

# Random trajectory
Xrand =  # TODO 
Urand = # TODO 

# Evaluate the cost
# t = prob.times[1]
# dt = prob.times[2]-prob.times[1]
# cost(prob.obj, Xrand, Urand)

# Evaluate the dynamics
# TODO

### DIRCOL Algorithm

In [5]:
# TASK: Complete implement DIRCOL Algo



backwardpass!

### Forward Pass

In [6]:
# TASK: Complete forward simulation of satelite 

forwardpass!

## Part (b): Simple trajectory 
Planar example with quibit interpolation and no Thurst conditions

In [1]:
# Task Solve Simple Trajectory

In [2]:
# vis = Visualizer()
# set_mesh!(vis, model)
# render(vis)

In [3]:
# visualize!(vis, model, tf, Xline)

## Part (c): Higher Order DIRCOL
Solve the simple planar problem with unconstrained thrust with higher order DIRCOL 3->8th order

In [4]:
# TASK: Implement the following method

In [5]:
# TASK: Plot the comparison


## Part (d): Inequality Constraints 
Using the thruster constraints solve the optimization problem

In [6]:
# TASK: Inequality constraints

In [7]:
# render(vis)

In [9]:
# visualize!(vis, model, tf, Xflip)

## Part (e): Non-Planar Satellite Control


In [10]:
# TASK: Non-Planar Control

## TESTS

In [11]:
# run_tests();